In [ ]:
# default_exp label_assistant

# LabelAssistant

API details

In [ ]:
# hide
from nbdev.showdoc import *
from fastcore.nb_imports import *
from fastcore.test import *

In [ ]:
# export
import os
import pandas as pd
import winkie.dlc_importer as dlc_importer
from winkie.dlc_importer import DLCImporter

class LabelAssistant:
    """Used to label data and generate labelled data.
    
    Please note that generating the labelled data will, as of now, also apply all pre-processing included in 
    `dlc_importer`.
    """

    def __init__(self, behaviors, data_path, default_label='not_defined'): 
        self.behaviors = behaviors
        self.default_label = default_label
        self.files = os.listdir(data_path)
        self.file_paths = [os.path.join(data_path, f) for f in self.files]
        self.labels = {'behavior': [], 'start': [], 'end': [], 'file': []}
        self.imp = DLCImporter()

    def addLabel(self, behavior, start_frame, end_frame, file_name):
        if behavior not in self.behaviors:
            raise ValueError(f"Unknown behaviour: {behavior}")
        if file_name not in self.files:
            raise FileNotFoundError(file_name)
        
        # There might be a user error of labellig frames that are not existent in the specified file.
        # We decide to ignore this potential user error for now, since we don't want to initially load the files.

        self.labels['behavior'].append(behavior)
        self.labels['start'].append(start_frame)
        self.labels['end'].append(end_frame)
        self.labels['file'].append(file_name)
    
    def apply_labels(self, origin_bodypart='body'):
        "Apply labels on all available data and execue preprocessing."
        
        def import_and_tag(file_name):
            "Note that this will also preprocess the `dlc_importer` coordinates."
            df = self.imp.import_hdf(file_name)
            # TODO: encapsulate sequence of steps into function in DLCImporter?
            df = dlc_importer.transform_to_relative(df, origin_bodypart)
            df = dlc_importer.add_middle_neck(df) # this is a bit opinionated
            df = dlc_importer.add_rotation(df)
            df = dlc_importer.apply_rotation(df)
            df['file_name'] = os.path.basename(file_name)
            df['frame'] = df.index
            
            return df

        df_list = [import_and_tag(f) for f in self.file_paths]
        df_merged = pd.concat(df_list)
        df_merged['behavior'] = self.default_label

        for i, f in enumerate(self.labels['file']):
            file_name_select = df_merged['file_name'] == f
            frame_select = (self.labels['start'][i] <= df_merged['frame']) & (df_merged['frame'] <= self.labels['end'][i])
            df_merged.loc[file_name_select & frame_select, ['behavior']] = self.labels['behavior'][i]

        return df_merged

    def to_df(self):
        return pd.DataFrame(self.labels)

In [ ]:
path_to_dlc_files = 'example_data'
behaviors = ['pecking', 'shaking']
default_label = 'not_defined'
ass = LabelAssistant(behaviors, path_to_dlc_files, default_label)
ass.addLabel('pecking', 2, 5, 'coordinates.h5')

# Make sure a user does not have typos during labelling
def _missing_label(): ass.addLabel('peckking', 100, 200, 'coordinates.h5')
with ExceptionExpected(ex=ValueError, regex="Unknown behaviour: peckking"): _missing_label()

def _missing_file(): ass.addLabel('pecking', 100, 200, 'non_existent_file.h5')
with ExceptionExpected(ex=FileNotFoundError, regex="non_existent_file.h5"): _missing_file()

ass.addLabel('shaking', 10, 15, 'coordinates.h5')
df = ass.to_df()

df_expected = pd.DataFrame({'behavior': ['pecking', 'shaking'], 'start': [2, 10], 'end': [5, 15], 'file': ['coordinates.h5', 'coordinates.h5']})
pd.testing.assert_frame_equal(df, df_expected)

df_applied = ass.apply_labels()
display(df_applied)

df_c1 = df_applied[df_applied['file_name'] == 'coordinates.h5']
df_c2 = df_applied[df_applied['file_name'] == 'coordinates2.h5']

df_c1_4 = df_c1[df_c1['frame'] == 4]
df_c2_4 = df_c2[df_c2['frame'] == 4]
test_eq(df_c1_4['behavior'].iloc[0], 'pecking')
test_eq(df_c2_4['behavior'].iloc[0], default_label)

df_c1_12 = df_c1[df_c1['frame'] == 12]
test_eq(df_c1_12['behavior'].iloc[0], 'shaking')

bodyparts       head                              beak                         \
coords             x           y likelihood          x           y likelihood   
0         -24.756667  120.508710   0.999999  19.177848  137.378264   0.999981   
1         -25.508326  114.511197   0.999999  18.443110  132.829171   0.999951   
2         -24.669384  119.007761   0.999999  19.299872  135.928651   0.999978   
3         -25.269855  119.743642   0.999999  19.277590  136.612289   0.999985   
4         -26.205736  120.223085   0.999999  19.378876  137.673975   0.999990   
..               ...         ...        ...        ...         ...        ...   
95         -5.439143  142.633716   1.000000  11.807338  150.765213   0.018886   
96         -5.111411  144.342239   1.000000  11.891120  152.534492   0.016135   
97         -4.902319  142.167785   1.000000  12.372827  149.214393   0.007289   
98         -5.977148  142.247955   1.000000  11.306788  149.369812   0.004682   
99         -6.258121  141.260104   1.000000  11.399601  148.452320   0.010126   

bodyparts  left_neck                        right_neck  ...       body  \
coords             x           y likelihood          x  ... likelihood   
0          37.599212   99.638046   0.999998 -37.599212  ...   0.999992   
1          37.685982   94.642246   0.999999 -37.685982  ...   0.999873   
2          38.162277   96.605391   0.999998 -38.162277  ...   0.999946   
3          38.326491   97.857522   0.999998 -38.326491  ...   0.999970   
4          37.930796   99.283034   0.999999 -37.930796  ...   0.999938   
..               ...         ...        ...        ...  ...        ...   
95         25.405257  103.331178   0.999998 -25.405257  ...   0.999999   
96         25.831407  104.212114   0.999999 -25.831407  ...   0.999999   
97         25.929811  102.332530   0.999999 -25.929811  ...   0.999997   
98         25.838286  103.199445   1.000000 -25.838286  ...   0.999997   
99         25.362818  103.041361   0.999999 -25.362818  ...   0.999992   

bodyparts       tail                          middle_neck              \
coords             x           y likelihood             x           y   
0          -4.411014 -152.478855   0.999999 -1.365713e-14   89.158479   
1           0.187954 -159.670477   0.999997 -1.359671e-15   83.503899   
2          -9.546297 -154.782895   0.999999  3.715442e-15   87.485969   
3          -7.593725 -153.269178   0.999998 -3.211908e-14   88.193103   
4          -7.305264 -152.220668   0.999998 -2.563278e-14   90.176009   
..               ...         ...        ...           ...         ...   
95         34.718510 -170.572513   1.000000  1.667139e-14  100.388462   
96         30.892145 -170.003613   1.000000 -4.067222e-15  102.665676   
97         28.892545 -174.796723   0.999999 -1.157706e-14  100.767981   
98         30.290261 -174.304594   0.999999 -2.139548e-14  101.112080   
99         29.300697 -174.067206   1.000000 -3.437060e-14  100.597551   

bodyparts rotation_angle        file_name frame     behavior  
coords                                                        
0            -116.026775   coordinates.h5     0  not_defined  
1            -117.149092   coordinates.h5     1  not_defined  
2            -115.283267   coordinates.h5     2      pecking  
3            -115.645195   coordinates.h5     3      pecking  
4            -115.797288   coordinates.h5     4      pecking  
..                   ...              ...   ...          ...  
95           -134.573393  coordinates2.h5    95  not_defined  
96           -134.105164  coordinates2.h5    96  not_defined  
97           -133.609448  coordinates2.h5    97  not_defined  
98           -134.109740  coordinates2.h5    98  not_defined  
99           -134.093794  coordinates2.h5    99  not_defined  

[200 rows x 42 columns]

It is then advised to save the labelled data for future processing.

In [ ]:
df_applied.to_hdf('_tmp_labelled_behaviors.h5', key='labels')
display(pd.read_hdf('_tmp_labelled_behaviors.h5'))

bodyparts       head                              beak                         \
coords             x           y likelihood          x           y likelihood   
0         -24.756667  120.508710   0.999999  19.177848  137.378264   0.999981   
1         -25.508326  114.511197   0.999999  18.443110  132.829171   0.999951   
2         -24.669384  119.007761   0.999999  19.299872  135.928651   0.999978   
3         -25.269855  119.743642   0.999999  19.277590  136.612289   0.999985   
4         -26.205736  120.223085   0.999999  19.378876  137.673975   0.999990   
..               ...         ...        ...        ...         ...        ...   
95         -5.439143  142.633716   1.000000  11.807338  150.765213   0.018886   
96         -5.111411  144.342239   1.000000  11.891120  152.534492   0.016135   
97         -4.902319  142.167785   1.000000  12.372827  149.214393   0.007289   
98         -5.977148  142.247955   1.000000  11.306788  149.369812   0.004682   
99         -6.258121  141.260104   1.000000  11.399601  148.452320   0.010126   

bodyparts  left_neck                        right_neck  ...       body  \
coords             x           y likelihood          x  ... likelihood   
0          37.599212   99.638046   0.999998 -37.599212  ...   0.999992   
1          37.685982   94.642246   0.999999 -37.685982  ...   0.999873   
2          38.162277   96.605391   0.999998 -38.162277  ...   0.999946   
3          38.326491   97.857522   0.999998 -38.326491  ...   0.999970   
4          37.930796   99.283034   0.999999 -37.930796  ...   0.999938   
..               ...         ...        ...        ...  ...        ...   
95         25.405257  103.331178   0.999998 -25.405257  ...   0.999999   
96         25.831407  104.212114   0.999999 -25.831407  ...   0.999999   
97         25.929811  102.332530   0.999999 -25.929811  ...   0.999997   
98         25.838286  103.199445   1.000000 -25.838286  ...   0.999997   
99         25.362818  103.041361   0.999999 -25.362818  ...   0.999992   

bodyparts       tail                          middle_neck              \
coords             x           y likelihood             x           y   
0          -4.411014 -152.478855   0.999999 -1.365713e-14   89.158479   
1           0.187954 -159.670477   0.999997 -1.359671e-15   83.503899   
2          -9.546297 -154.782895   0.999999  3.715442e-15   87.485969   
3          -7.593725 -153.269178   0.999998 -3.211908e-14   88.193103   
4          -7.305264 -152.220668   0.999998 -2.563278e-14   90.176009   
..               ...         ...        ...           ...         ...   
95         34.718510 -170.572513   1.000000  1.667139e-14  100.388462   
96         30.892145 -170.003613   1.000000 -4.067222e-15  102.665676   
97         28.892545 -174.796723   0.999999 -1.157706e-14  100.767981   
98         30.290261 -174.304594   0.999999 -2.139548e-14  101.112080   
99         29.300697 -174.067206   1.000000 -3.437060e-14  100.597551   

bodyparts rotation_angle        file_name frame     behavior  
coords                                                        
0            -116.026775   coordinates.h5     0  not_defined  
1            -117.149092   coordinates.h5     1  not_defined  
2            -115.283267   coordinates.h5     2      pecking  
3            -115.645195   coordinates.h5     3      pecking  
4            -115.797288   coordinates.h5     4      pecking  
..                   ...              ...   ...          ...  
95           -134.573393  coordinates2.h5    95  not_defined  
96           -134.105164  coordinates2.h5    96  not_defined  
97           -133.609448  coordinates2.h5    97  not_defined  
98           -134.109740  coordinates2.h5    98  not_defined  
99           -134.093794  coordinates2.h5    99  not_defined  

[200 rows x 42 columns]